In [2]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))

# data_dir = join(parent_dir, 'data')
# data_file = join(data_dir, "data.csv")
# docs_dir = join(parent_dir, 'docs')
# if exists(docs_dir):
#     doc_file = relpath(join(docs_dir, 'practical_works_linear_programing_v3.pdf'))
#     IFrame(doc_file, width=1200, height=350)
# matrix = csv_to_matrix(data_file)
parent_dir

NameError: name 'abspath' is not defined